<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100" align="right" /> 

# Cours TAL - Laboratoire 4<br/>Reconnaissance des entités nommées

**Objectif**

L'objectif de ce labo est de comparer la reconnaissance des entités nommées (*named entity recognition*, NER) faite par spaCy avec celle faite par NLTK, sur des données en anglais fournies sur Cyberlearn.  Veuillez fournir les scores de rappel, précision et F1-score pour chacun des tags présents dans les données de test.  Veuillez comparer deux modèles de spaCy, 'en_core_web_sm' et 'en_core_web_lg'.

Vous pouvez concevoir l'ensemble du projet par vous-mêmes, ou suivre les indications suivantes.

## 1. NER avec spaCy et NLTK sur un texte court

In [1]:
import spacy
from spacy.tokens import Doc

In [2]:
# !python -m spacy download en_core_web_sm 
# exécuter la ligne ci-dessus une fois, si nécessaire

In [3]:
nlp = spacy.load("en_core_web_sm")

In [4]:
raw_text = "Reinhold Messner made a solo ascent of Mount Everest and was later a member of the European Parliament." 

**1a.** Veuillez traiter ce texte avec la pipeline 'nlp', et pour chaque entité nommée trouvée veuillez afficher les mots qui la composent et son type.

In [5]:
docs = nlp(raw_text)

for ent in docs.ents:
    print(ent.text + " [" + ent.label_ + "]")

Reinhold Messner [PERSON]
Mount Everest [LOC]
the European Parliament [ORG]


In [6]:
import nltk
#nltk.download('maxent_ne_chunker') 
#nltk.download('words') 
# exécuter les deux lignes ci-dessus une fois, si nécessaire

**1b.** Veuillez effectuer avec NLTK la tokenization, le POS tagging et le *NE chunking* (voir la [documentation NLTK](https://www.nltk.org/api/nltk.chunk.ne_chunk.html#nltk.chunk.ne_chunk)).  Veuillez afficher le résultat et indiquer son type.

In [29]:

tokens = nltk.word_tokenize(raw_text)
pos_tagged_tokens = nltk.pos_tag(tokens)    
ne_chunks = nltk.ne_chunk(pos_tagged_tokens)


**1c.** Veuillez afficher, pour chaque entité nommée, les mots qui la composent et son type.  Vous pouvez parcourir le résultat précédent avec une boucle for, et déterminer si un noeud a une étiquette avec la fonction `hasattr(noeud, 'label')`.

In [35]:
for ne_chunk in ne_chunks:
    if hasattr(ne_chunk, 'label'):
        print(ne_chunk)

(PERSON Reinhold/NNP)
(PERSON Messner/NNP)
(PERSON Mount/NNP Everest/NNP)
(ORGANIZATION European/NNP Parliament/NNP)


**1d.** À ce stade, que pensez-vous de la qualité des résultats de chaque système ?

In [9]:
# Sur cet phrase en particulier, nlp s'en sort bien mieux, il a fait tous juste.
# Il a identifié "Reinhold Messner" comme une personne, le mont everest comme une location et le parlement européen comme une organisation.
# Alors que nltk a identifié "Reinhold Messner" comme 2 personnes différentes, le mont everest comme une personne. Il a juste fait juste le parlement européen

## 2. Prise en main des données de test

**2a.** Quel est le format du fichier `eng.test.a.conll` ?  Quelle information contient chaque colonne ?  Quel est le format des tags NE ?

Note : ce fichier fait partie des données de test pour la NER sur l'anglais de la conférence [CoNLL](https://www.clips.uantwerpen.be/pages/past-workshops) 2003. On peut lire [ici](https://www.clips.uantwerpen.be/conll2003/ner/) la description de la tâche et les scores obtenus.  On peut trouver une copie des données [ici](https://sourceforge.net/p/text-analysis/svn/1243/tree/text-analysis/trunk/Corpora/CoNLL/2003/) ou [ici](https://github.com/synalp/NER/tree/master/corpus/CoNLL-2003).  Les textes proviennent du [corpus Reuters](http://trec.nist.gov/data/reuters/reuters.html).

In [10]:
# Format IO (pas de beginning)
# Colonne 1 : Le mot
# Colonne 2 : Le POS tag
# Colonne 3 : Le chunk NE
# Colonne 4 : Le label NE

**2b.** Veuillez charger les données de `eng.test.a.conll` grâce à la classe `ConllCorpusReader` de NLTK vue dans les labos précédents (voir [documentation](https://www.nltk.org/api/nltk.corpus.reader.conll.html#nltk.corpus.reader.conll.ConllCorpusReader)). Veuillez lire les colonnes qui contiennent les tokens ('words'), les POS tags ('pos') et les informations sur les entités nommées ('chunk') et afficher les trois premières phrases, accessibles via la méthode `.iob_sents()`.

In [11]:
from nltk.corpus.reader.conll import ConllCorpusReader

corpus = ConllCorpusReader('.', 'eng.test.a.conll',columntypes=['words', 'pos', 'ignore','chunk'])

sents = corpus.iob_sents()
print(sents[0])
print(sents[1])
print(sents[2])

#TODO pas sur pourquoi mais y'a rien dans la première phrase

[]
[('CRICKET', 'NNP', 'O'), ('-', ':', 'O'), ('LEICESTERSHIRE', 'NNP', 'I-ORG'), ('TAKE', 'NNP', 'O'), ('OVER', 'IN', 'O'), ('AT', 'NNP', 'O'), ('TOP', 'NNP', 'O'), ('AFTER', 'NNP', 'O'), ('INNINGS', 'NNP', 'O'), ('VICTORY', 'NN', 'O'), ('.', '.', 'O')]
[('LONDON', 'NNP', 'I-LOC'), ('1996-08-30', 'CD', 'O')]


**2c.** Veuillez préparer les données pour le test, en ne gardant que les phrases ayant au moins trois (3) tokens (pas 0, 1, 2) :

* une variable `test_tokens` contiendra les tokens groupés par phrase (liste de listes de strings)
* une variable `test_tags` contiendra les tags NE en une seule liste (en vue de l'évaluation)

In [78]:
sents = list(sents)
sents = list(filter(lambda x: len(x) > 2, sents))

test_tokens = []
test_tags = []
for sent in sents:
    curr_tokens = []
    for token in sent:
       curr_tokens.append(token[0])
       test_tags.append(token[2])
    test_tokens.append(curr_tokens)

print(test_tokens[0])
print(len(test_tags))

['CRICKET', '-', 'LEICESTERSHIRE', 'TAKE', 'OVER', 'AT', 'TOP', 'AFTER', 'INNINGS', 'VICTORY', '.']
50817


**2d.** Combien d'occurrences de tags contient `test_tags`?  Combien de tags différents y a-t-il, et lesquels sont-ils ?  Combien il y a d'occurrences de tags de chaque type ?  Combien de phrases y a-t-il dans `test_tokens` ?

In [80]:
unique_data = set(test_tags)
print("Nombre de phrases dans test_tokens : ",len(test_tokens))
print("Nombre de tags dans test_tags : ",len(test_tags)) 
print("Nombre de tags uniques : ",len(unique_data))
print("List des tags : " , unique_data)
print("Nombre d'occurence de O",test_tags.count("O"))
print("Nombre d'occurence de I-PER",test_tags.count("I-PER"))
print("Nombre d'occurence de I-LOC",test_tags.count("I-LOC"))
print("Nombre d'occurence de I-MISC",test_tags.count("I-MISC"))
print("Nombre d'occurence de I-ORG",test_tags.count("I-ORG"))



Nombre de phrases dans test_tokens :  2970
Nombre de tags dans test_tags :  50817
Nombre de tags uniques :  5
List des tags :  {'O', 'I-MISC', 'I-ORG', 'I-LOC', 'I-PER'}
Nombre d'occurence de O 42474
Nombre d'occurence de I-PER 3097
Nombre d'occurence de I-LOC 1938
Nombre d'occurence de I-MISC 1228
Nombre d'occurence de I-ORG 2080


## 3. Performances de NLTK pour la NER

**3a.** Le NER de NLTK a un jeu de tags différents de celui des données de test.  Veuillez chercher les informations pour compléter la fonction suivante qui convertir chaque tag du NER de NLTK dans le tag correspondant pour les données de test.  Attention à la logique des conversions. 

In [81]:
def convert_nltk_conll(nltk_tag):
   translated_tags = []
   corresp = { 'O':'O',
               'ORGANIZATION':'I-ORG',
               'PERSON':'I-PER',
               'LOCATION':'I-LOC',
               'DATE': 'I-MISC',
               'TIME': 'I-MISC',
               'MONEY': 'I-MISC',	
               'PERCENT': 'I-MISC',
               'FACILITY': 'I-LOC',
               'GPE': 'I-LOC'
             }
   for tag in nltk_tag:
       if tag in corresp:
         translated_tags.append(corresp[tag])
   return translated_tags


**3b.** Veuillez exécuter la NER de NLTK sur chacune des phrases de `test_tokens`, ce qui assure que NLTK aura la même tokenisation que les données de référence.  Veuillez stocker les tags dans une liste unique appelée `nltk_tags`.

In [72]:
nltk_tags = []
nb_token = 0
nb_tagged_token = 0
nb_ne_chunks = 0
for sent in test_tokens:
    nb_token += len(sent)
    pos_tagged_tokens = nltk.pos_tag(sent) 
    nb_tagged_token += len(pos_tagged_tokens)   
    ne_chunks = nltk.ne_chunk(pos_tagged_tokens)
    nb_ne_chunks += len(ne_chunks)
    for ne_chunk in ne_chunks:
        if hasattr(ne_chunk, 'label'):
            nltk_tags.append(ne_chunk.label())
            for leaf in ne_chunk.leaves():
                 if hasattr(ne_chunk, 'label'):
                    nltk_tags.append(leaf.label())
                else:
                    nltk_tags.append('O')
        else:
            nltk_tags.append('O')
print(nltk_tags)


['GPE', 'NNP', 'O', 'ORGANIZATION', 'NNP', 'O', 'O', 'O', 'ORGANIZATION', 'NNP', 'O', 'ORGANIZATION', 'NNP', 'O', 'O', 'GPE', 'NNP', 'GPE', 'JJ', 'O', 'PERSON', 'NNP', 'NNP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'ORGANIZATION', 'NNP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PERSON', 'NNP', 'O', 'ORGANIZATION', 'NNP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PERSON', 'NNP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'GPE', 'NNP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'FACILITY', 'NNP', 'NNP', 'O', 'PERSON', 'NNP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'GPE', 'NNP', 'O', 'PERSON', 'NNP', 'NNP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PERSON', 'NNP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'PERSON', 'NNP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'GPE', 'NN', 'O', 'O', 'O', 'O', 'O'

**3c.** Veuillez convertir les tags de `nltk_tags` grâce à la fonction précédente, dans une liste appelée `nltk_tags_conv`.  Veuillez afficher le nombre total de tags et les dix premiers.  Vous pouvez plusieurs essais en changeant la fonction, pour aboutir à la conversion qui maximise le score.

In [75]:
nltk_tags_conv = convert_nltk_conll(nltk_tags)
print(nltk_tags_conv)
print(test_tags)

['I-LOC', 'O', 'I-ORG', 'O', 'O', 'O', 'I-ORG', 'O', 'I-ORG', 'O', 'O', 'I-LOC', 'I-LOC', 'O', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-PER', 'O', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'O', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'O', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-PER', 'O', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'I-PER', 'O', 'I-ORG', 'O', 'I-LOC', 'O', 'O', 'O', 'O', 'I-LOC', 'O', 'O', 'O', 'O',

**3d.** Veuillez afficher le rapport d'évaluation de classification obtenu de Scikit-learn et la matrice de confusion pour tous les types de tags apparaissant dans les données de test.

In [45]:
from sklearn.metrics import classification_report, confusion_matrix

In [76]:
# Afficher le rapport d'évaluation de classification
print("Rapport d'évaluation de classification :")
print(classification_report(nltk_tags_conv, test_tags))

# Afficher la matrice de confusion
print("Matrice de confusion :")
print(confusion_matrix(nltk_tags_conv, test_tags))

Rapport d'évaluation de classification :


c:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python39\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       I-LOC       0.06      0.04      0.05      2397
      I-MISC       0.00      0.00      0.00         0
       I-ORG       0.02      0.04      0.03      1084
       I-PER       0.05      0.08      0.06      1895
           O       0.89      0.84      0.86     43776

    accuracy                           0.75     49152
   macro avg       0.20      0.20      0.20     49152
weighted avg       0.80      0.75      0.78     49152

Matrice de confusion :
[[  107    53    84   208  1945]
 [    0     0     0     0     0]
 [   45    26    45    73   895]
 [   90    52    83   143  1527]
 [ 1644  1047  1818  2601 36666]]


In [ ]:
#TODO PROBLEM AVEC LE NOMBRE DE TOKEN JSP CKISPASSE

## 4. Performances de spaCy pour la NER

**4a.** Le NER de spaCy a aussi un jeu de tags différents de celui des données de test.  Veuillez chercher les informations pour compléter la fonction suivante qui convertir chaque tag du NER de spaCy dans le tag correspondant pour les données de test.  Attention à la logique des conversions. 

In [16]:
# def convert_spacy_conll(spacy_tag):


**4b.** Veuillez exécuter la NER de spaCy sur chacune des phrases de `test_tokens`, ce qui assure que spaCy aura la même tokenisation que les données de référence.  Veuillez stocker les tags dans une liste unique appelée `spacy_tags`.

**4c.** Veuillez convertir les tags de `spacy_tags` grâce à la fonction précédente, dans une liste appelée `spacy_tags_conv`.  Veuillez afficher le nombre total de tags et les dix premiers.  Vous pouvez plusieurs essais en changeant la fonction, pour aboutir à la conversion qui maximise le score.

**4d.** Veuillez afficher le rapport d'évaluation de classification obtenu de Scikit-learn et la matrice de confusion pour tous les types de tags apparaissant dans les données de test.

**4e.** Veuillez exécuter également le modèle 'en_core_web_lg' de spacy et afficher le rapport d'évaluation (mais pas la matrice de confusion).  Vous pouvez recopier ici le minimum de code nécessaire à l'obtention des résultats, avec une nouvelle pipeline spaCy appelée 'nlp2'.

## 5. Discussion finale

Veuillez comparer les scores des trois modèles testés, en termes de **macro avg**.  Pourquoi ce score est-il le plus informatif ?  Veuillez indiquer également la taille des modèles spaCy évalués.

**Fin du Labo.** Veuillez nettoyer ce notebook en gardant seulement les résultats désirés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.